In [1]:
from __future__ import print_function

import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

seed = 2021
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)

# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)
    
    
class Dataset(data.Dataset):
    def __init__(self, X1,Y):
        self.X1 = X1
        self.Y = Y
    def __len__(self):        
        return len(self.X1)
    
    def __getitem__(self, index):
        x = self.X1[index]
        y = self.Y[index]
        return x,y

#from model import Attention, GatedAttention

## Negetive pictures

In [2]:
import os
def readFeatures(path):
    features=np.array([])
    start=0
    for file in os.listdir(path):
        d = os.path.join(path, file)
        datafile=d+"/features.npy"
        try:
            temp=np.load(datafile)
#             print(datafile)
#             print(temp.shape)
            if(start==0):
                features=temp
                if(d[-10]=='S'):
                    label=np.ones(len(temp))
                else:
                    label=np.zeros(len(temp))
                start=1
            else:
                features=np.vstack((features,temp))
                if(d[-10]=='S'):
                    label=np.append(label,np.ones(len(temp)))
                else:
                    label=np.append(label,np.zeros(len(temp)))
        except IOError:
            print('file not in Scripted')

            
    return features,label

def load_data(path,mode):
    if mode == "test":
        xTest,yTest=readFeatures(path+"/"+mode)
        xTest = xTest.reshape(len(xTest),1,19,24)
        return Dataset(xTest,yTest)
        
    elif mode == "train":
        xTrain,yTrain=readFeatures(path+"/"+mode)
        xTrain = xTrain.reshape(len(xTrain),1,19,24)
        return Dataset(xTrain,yTrain)
        
    elif mode == "val":
        xVal,yVal=readFeatures(path+"/"+mode)
        xVal = xVal.reshape(len(xVal),1,19,24)
        return Dataset(xVal,yVal)
    else:
        print("Mode not defined")
        return

traindataset = load_data("../Participant_wise",'train')
testdataset = load_data('../Participant_wise','test')
valdataset = load_data('../Participant_wise','val')

In [3]:
# xTrain=np.load("../extracted_features/0/GRP-3/input_features_train_grp3.npy")
# labels_stress_train=np.load("../extracted_features/0/GRP-3/labels_stress_train_grp3.npy")
# yTrain=np.load("../extracted_features/0/GRP-3/labels_train_grp3.npy")

# xTest=np.load("../extracted_features/0/GRP-3/input_features_test_grp3.npy")
# labels_stress_test=np.load("../extracted_features/0/GRP-3/labels_stress_test_grp3.npy")
# yTest=np.load("../extracted_features/0/GRP-3/labels_test_grp3.npy")


# xVal=np.load("../extracted_features/0/GRP-3/input_features_val_grp3.npy")
# labels_stress_val=np.load("../extracted_features/0/GRP-3/labels_stress_val_grp3.npy")
# yVal=np.load("../extracted_features/0/GRP-3/labels_val_grp3.npy")


In [4]:
# xTrain.shape

In [5]:
# xVal.shape

In [6]:
# xTest.shape

## Baseline 

In [7]:
# train_features=np.load("../extracted_features/bl/0/GRP-3/input_features_train_grp3.npy")
# labels_stress_train=np.load("../extracted_features/bl/0/GRP-3/labels_stress_train_grp3.npy")
# labels_train=np.load("../extracted_features/bl/0/GRP-3/labels_train_grp3.npy")

# test_features=np.load("../extracted_features/bl/0/GRP-3/input_features_test_grp3.npy")
# labels_stress_test=np.load("../extracted_features/bl/0/GRP-3/labels_stress_test_grp3.npy")
# labels_test=np.load("../extracted_features/bl/0/GRP-3/labels_test_grp3.npy")

# val_features=np.load("../extracted_features/bl/0/GRP-3/input_features_val_grp3.npy")
# labels_stress_val=np.load("../extracted_features/bl/0/GRP-3/labels_stress_val_grp3.npy")
# labels_val=np.load("../extracted_features/bl/0/GRP-3/labels_val_grp3.npy")

In [8]:
# print(train_features.shape,labels_train.shape)
# print(test_features.shape,labels_test.shape)
# print(val_features.shape,labels_val.shape)

(497, 19, 24) (497,)
(119, 19, 24) (119,)
(110, 19, 24) (110,)


In [9]:
# train_features=train_features.reshape(len(train_features),1,19,24)
# test_features=test_features.reshape(len(test_features),1,19,24)
# val_features=val_features.reshape(len(val_features),1,19,24)

In [10]:
# xTrain = xTrain.reshape(len(xTrain),1,19,24)
# xTest = xTest.reshape(len(xTest),1,19,24)
# xVal = xVal.reshape(len(xVal),1,19,24)

In [11]:
# xTrain=np.append(train_features,xTrain,axis=0)
# xTest=np.append(test_features,xTest,axis=0)
# xVal=np.append(val_features,xVal,axis=0)
# yTrain=np.append(labels_train,yTrain,axis=0)
# yTest=np.append(labels_test,yTest,axis=0)
# yVal=np.append(labels_val,yVal,axis=0)

In [12]:
# print(yTest)

In [13]:
# print(xTrain.shape,yTrain.shape)
# print(xTest.shape,yTest.shape)
# print(xVal.shape,yVal.shape)

In [14]:
# traindataset = Dataset(train_features,labels_train)
# testdataset = Dataset(test_features,labels_test)
# valdataset = Dataset(val_features,labels_val)

# traindataset = Dataset(xTrain,yTrain)
# testdataset = Dataset(xTest,yTest)
# valdataset = Dataset(xVal,yVal)

In [15]:
class Stammering(data_utils.Dataset):
    def __init__(self, target_number=1, mean_bag_length=5, var_bag_length=2, num_bag=150, seed=2021, train="train"):
        self.target_number = target_number
        self.mean_bag_length = mean_bag_length
        self.var_bag_length = var_bag_length
        self.num_bag = num_bag
        self.train = train
        self.r = np.random.RandomState(seed)

        self.num_in_train = 512
        self.num_in_test = 122
        self.num_in_val=118

        if self.train=="train":
            self.train_bags_list, self.train_labels_list = self._create_bags()
        elif self.train=="val":
            self.val_bags_list, self.val_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _create_bags(self):
        if self.train=="train":
            print("train")
            loader = data_utils.DataLoader(traindataset,
                                           batch_size=1,
                                           shuffle=True)
        elif self.train=="val":
            print("val")
            loader = data_utils.DataLoader(valdataset,
                                           batch_size=1,
                                           shuffle=True)
        else:
            loader = data_utils.DataLoader(testdataset,
                                           batch_size=1,
                                           shuffle=True)
            
        bags_list = []
        labels_list = []
        for (batch_data, batch_labels) in loader:
            #print(batch_data.shape)
            bags_list.append(batch_data.reshape(19,1,24))
            temp = torch.as_tensor(np.array([batch_labels for x in range(19)]))
            labels_list.append(temp)
            
               
        #print(bags_list)
        #print(labels_list)
        return bags_list, labels_list

    def __len__(self):
        if self.train=="train":
            return len(self.train_labels_list)
        elif self.train=="val":
            return len(self.val_labels_list)
        else:
            return len(self.test_labels_list)

    def __getitem__(self, index):
        if self.train=="train":
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]), self.train_labels_list[index]]
        elif self.train=="val":
            bag = self.val_bags_list[index]
            label = [max(self.val_labels_list[index]), self.val_labels_list[index]]
        else:
            bag = self.test_bags_list[index]
            label = [max(self.test_labels_list[index]), self.test_labels_list[index]]

        return bag, label

In [16]:
train_loader = data_utils.DataLoader(Stammering(train="train"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
test_loader = data_utils.DataLoader(Stammering(train="test"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
val_loader = data_utils.DataLoader(Stammering(train="val"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)

train
val


In [17]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 128
        self.D = 64
        self.K = 1

        self.feature_extractor_part1 = nn.Sequential(
            nn.Linear(24,200),
            nn.ReLU(),
            nn.Conv1d(1, 20, kernel_size=2),
            nn.ReLU(),
            nn.Linear(199,64),
            nn.ReLU(),
        )

        self.feature_extractor_part2 = nn.Sequential(
            nn.BatchNorm1d(1280, affine=False),
            nn.Linear(1280, self.L),
            nn.ReLU(),
        )

        self.attention = nn.Sequential(
            nn.Linear(self.L, self.D),
            nn.Tanh(),
            nn.Linear(self.D, self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x=x.squeeze(0)

        H = self.feature_extractor_part1(x)
        
        #H = H.view(-1, 256*5*4)
        H = H.view(H.size(0), -1)
        #print(H.shape)
        H = self.feature_extractor_part2(H)  # NxL

        A = self.attention(H)  # NxK

        A = torch.transpose(A, 1, 0)  # KxN

        A = F.softmax(A, dim=1)  # softmax over N

        M = torch.mm(A, H)  # KxL

        Y_prob = self.classifier(M)
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat, A



    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat, _ = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _, A = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        neg_log_likelihood = -1. * (Y * torch.log(Y_prob) + (1. - Y) * torch.log(1. - Y_prob))  # negative log bernoulli

        return neg_log_likelihood, A

In [18]:
model = Attention()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

In [19]:
# from torchsummary import summary
# summary(model.cuda(),(1,24))
# torch.save(model, "modelarchitect_attn.pt")

In [20]:
if torch.cuda.is_available():model.cuda()

In [21]:
best_model = "./saved_models/best_model_scripted_Attention_MIL_raw"
train_acc = []
modelname=[]
val_acc = []
best_acc = 0
for epoch in range(0, 15): #15 epochs seed =2021,11
    model.train()
    train_loss = 0.
    train_error = 0.
    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(train_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        optimizer.zero_grad()
        loss, _ = model.calculate_objective(data.float(), bag_label.float())
        train_loss += loss.data[0]
        error,y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        train_error += error
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
        loss.mean().backward()
        optimizer.step()
    
    trainacc = accuracy_score(y,ypred)
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print("EPOCH ",epoch)
    print('Train : Loss: {:.4f}, Train error: {:.4f}, Train acc : {}'.format(train_loss.cpu().numpy()[0], 
                                                                                train_error,trainacc))
    y =[]
    ypred = []
    val_error=0.
    val_loss= 0.
    model.eval()
    for batch_idx, (data, label) in enumerate(val_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss, _ = model.calculate_objective(data.float(), bag_label.float())
        val_loss += loss.data[0]
        error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        val_error += error
        y_pred=y_pred.squeeze(1)

        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
    valacc = accuracy_score(y,ypred)
    val_loss /= len(val_loader)
    val_error /= len(val_loader)
    print('Val : Loss: {:.4f}, val error: {:.4f}, Val acc :{}'.format(val_loss.cpu().numpy()[0], val_error,valacc))
    if valacc>=best_acc:
        print("---------State saved---------")
        best_acc = valacc
        best_state=model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(epoch)+".pth")
        modelname.append(best_model+'_epoch_'+str(epoch)+".pth")
    print('Best validation accuracy ',best_acc)
        

EPOCH  0
Train : Loss: 0.6846, Train error: 0.4322, Train acc : 0.5678294573643411
Val : Loss: 0.6689, val error: 0.3898, Val acc :0.6101694915254238
---------State saved---------
Best validation accuracy  0.6101694915254238
EPOCH  1
Train : Loss: 0.6784, Train error: 0.4360, Train acc : 0.563953488372093
Val : Loss: 0.6816, val error: 0.4068, Val acc :0.5932203389830508
Best validation accuracy  0.6101694915254238
EPOCH  2
Train : Loss: 0.6536, Train error: 0.3992, Train acc : 0.6007751937984496
Val : Loss: 0.5808, val error: 0.3644, Val acc :0.635593220338983
---------State saved---------
Best validation accuracy  0.635593220338983
EPOCH  3
Train : Loss: 0.6256, Train error: 0.3643, Train acc : 0.6356589147286822
Val : Loss: 0.7372, val error: 0.4492, Val acc :0.5508474576271186
Best validation accuracy  0.635593220338983
EPOCH  4
Train : Loss: 0.5646, Train error: 0.3023, Train acc : 0.6976744186046512
Val : Loss: 0.6893, val error: 0.3898, Val acc :0.6101694915254238
Best validatio

In [22]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_best(stress)_F1_91.pth")
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss, attention_weights = model.calculate_objective(data.float(), bag_label.float())
    test_loss += loss.data[0]
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
    
acc=accuracy_score(y,ypred)
# print(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
print(acc,f1score,precision,recall,roc,specificity)


0.9322033898305084 0.9298245614035087 0.9464285714285714 0.9137931034482759 0.9318965517241379 0.95


## Window-1

In [23]:
device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_best(stress)_F1_91.pth")
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
# xTest=np.load("../extracted_features/0/win-1/GRP-3/input_features_val_grp3.npy")
# yTest=np.load("../extracted_features/0/win-1/GRP-3/labels_val_grp3.npy")
# testdataset = Dataset(xTest,yTest)
testdataset = load_data('../Participant_wise_win-1','test')
test_loader = data_utils.DataLoader(Stammering(train="test"),worker_init_fn=seed_worker,batch_size=1,shuffle=True)
attn_weights=[]
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss, attention_weights = model.calculate_objective(data.float(), bag_label.float())
    attn_weights.append(np.argmax(attention_weights.cpu().detach().numpy() ))
    test_loss += loss.data[0]
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
acc=accuracy_score(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
# print(best_model)
print(acc,f1score,precision,recall,roc,specificity)

0.9322033898305084 0.9310344827586207 0.9310344827586207 0.9310344827586207 0.932183908045977 0.9333333333333333


## Window-2

In [24]:
device = torch.device("cuda")
model=Attention()
# best_state=torch.load("../best-models/Attention_MIL_best(stress)_F1_91.pth")
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
# xTest=np.load("../extracted_features/0/win-2/GRP-3/input_features_val_grp3.npy")
# yTest=np.load("../extracted_features/0/win-2/GRP-3/labels_val_grp3.npy")
# testdataset = Dataset(xTest,yTest)
testdataset = load_data('../Participant_wise_win-2','test')
test_loader = data_utils.DataLoader(Stammering(train="test"),worker_init_fn=seed_worker,batch_size=1,shuffle=True)
attn_weights=[]
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss, attention_weights = model.calculate_objective(data.float(), bag_label.float())
    attn_weights.append(np.argmax(attention_weights.cpu().detach().numpy() ))
    test_loss += loss.data[0]
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
acc=accuracy_score(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
# print(best_model)
print(acc,f1score,precision,recall,roc,specificity)

0.9322033898305084 0.9285714285714286 0.9629629629629629 0.896551724137931 0.9316091954022988 0.9666666666666667


In [25]:
print(y,ypred)

[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0] [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]
